Решенные сложные задачи с курса Симулятор **SQL**

1 МОДУЛЬ

In [ ]:
"Посчитайте общее количество заказов в таблице orders, количество заказов с пятью и более товарами и найдите долю заказов с пятью и более товарами в общем количестве заказов.
"В результирующей таблице отразите все три значения — поля назовите соответственно orders, large_orders, large_orders_share.
"Долю заказов с пятью и более товарами в общем количестве товаров округлите до двух знаков после запятой.
"Поля в результирующей таблице: orders, large_orders, large_orders_share"
SELECT count(order_id) as orders,
       count(order_id) filter (WHERE array_length(product_ids, 1) >= 5) as large_orders,
       round(count(order_id) filter (WHERE array_length(product_ids, 1) >= 5)/cast(count(order_id) as decimal),
             2) as large_orders_share
FROM   orders

In [ ]:
"Посчитайте, сколько пользователей никогда не отменяли свой заказ. Для этого из общего числа всех уникальных пользователей отнимите число уникальных пользователей, которые хотя бы раз отменяли заказ. Подумайте, какое условие необходимо указать в FILTER, чтобы получить корректный результат.
"Полученный столбец назовите users_count.
"Поле в результирующей таблице: users_count
SELECT count(distinct user_id)-count(distinct user_id) filter(WHERE action in ('cancel_order')) as users_count
FROM   user_actions

In [ ]:
"Для каждого дня недели в таблице user_actions посчитайте:

"Общее количество оформленных заказов.
"Общее количество отменённых заказов.
"Общее количество неотменённых заказов (т.е. доставленных).
"Долю неотменённых заказов в общем числе заказов (success rate).
"Новые колонки назовите соответственно created_orders, canceled_orders, actual_orders и success_rate. Колонку с долей неотменённых заказов округлите до трёх знаков после запятой.
"Все расчёты проводите за период с 24 августа по 6 сентября 2022 года включительно, чтобы во временной интервал попало равное количество разных дней недели.
"Группы сформируйте следующим образом: выделите день недели из даты с помощью функции to_char с параметром 'Dy', также выделите порядковый номер дня недели с помощью функции DATE_PART с параметром 'isodow'. Далее сгруппируйте данные по двум полям и проведите все необходимые расчёты.
"В результате должна получиться группировка по двум колонкам: с порядковым номером дней недели и их сокращёнными наименованиями.
"Результат отсортируйте по возрастанию порядкового номера дня недели.
"Поля в результирующей таблице: weekday_number, weekday, created_orders, canceled_orders, actual_orders, success_rate"
SELECT weekday_number,
       weekday,
       created_orders,
       canceled_orders,
       created_orders-canceled_orders as actual_orders,
       round((created_orders-canceled_orders)/(cast(created_orders as decimal)),
             3) as success_rate
FROM   (SELECT cast(date_part('isodow', time) as varchar) as weekday_number,
               to_char(time, 'Dy') as weekday,
               count(order_id) filter(WHERE action = 'create_order') as created_orders ,
               count(order_id) filter(WHERE action = 'cancel_order') as canceled_orders
        FROM   user_actions
        WHERE  time >= '2022-08-24 00:00:00'
           and time <= '2022-09-06 23:59:59'
        GROUP BY to_char(time, 'Dy'), date_part('isodow', time)) as koko
ORDER BY 1 asc

In [ ]:
"Для каждого пользователя в таблице user_actions посчитайте общее количество оформленных заказов и долю отменённых заказов.
"Новые колонки назовите соответственно orders_count и cancel_rate. Колонку с долей отменённых заказов округлите до двух знаков после запятой.
"В результат включите только тех пользователей, которые оформили больше трёх заказов и у которых показатель cancel_rate составляет не менее 0.5.
"Результат отсортируйте по возрастанию id пользователя.
"Поля в результирующей таблице: user_id, orders_count, cancel_rate"
SELECT user_id,
       round(count(distinct order_id) filter(WHERE action = 'cancel_order')/cast(count(distinct order_id) as decimal),
             2) as cancel_rate
FROM   user_actions
GROUP BY user_id
ORDER BY 1 asc

In [ ]:
"Разбейте пользователей из таблицы users на 4 возрастные группы:
"от 19 до 24 лет;
"от 25 до 29 лет;
"от 30 до 35 лет;
"от 36 до 41 года.
"Посчитайте число пользователей, попавших в каждую возрастную группу. Группы назовите соответственно «19-24», «25-29», «30-35», «36-41» (без кавычек).
"Выведите наименования групп и число пользователей в них. Колонку с наименованием групп назовите group_age, а колонку с числом пользователей — users_count.
"Отсортируйте полученную таблицу по колонке с наименованием групп по возрастанию.
"Поля в результирующей таблице: group_age, users_count"
SELECT case when date_part('year', age(current_date, birth_date)) between 19 and
                 24 then '19-24'
            when date_part('year', age(current_date, birth_date)) between 25 and
                 29 then '25-29'
            when date_part('year', age(current_date, birth_date)) between 30 and
                 35 then '30-35'
            when date_part('year', age(current_date, birth_date)) between 36 and
                 41 then '36-41' end as group_age,
       count(user_id) as users_count
FROM   users
WHERE  user_id is not null
GROUP BY group_age
ORDER BY 1 asc limit 4

In [ ]:
"По данным из таблицы orders рассчитайте средний размер заказа по выходным и по будням.
"Группу с выходными днями (суббота и воскресенье) назовите «weekend», а группу с будними днями (с понедельника по пятницу) — «weekdays» (без кавычек).
"В результат включите две колонки: колонку с группами назовите week_part, а колонку со средним размером заказа — avg_order_size.
"Средний размер заказа округлите до двух знаков после запятой.
"Результат отсортируйте по колонке со средним размером заказа — по возрастанию.
"Поля в результирующей таблице: week_part, avg_order_size"
SELECT case when date_part('dow', creation_time) in (0, 6) then 'weekend'
            else 'weekdays' end as week_part,
       round(avg(array_length(product_ids, 1)), 2) as avg_order_size
FROM   orders
GROUP BY week_part
ORDER BY 2 asc

In [ ]:
"Из таблицы orders выведите id и содержимое заказов, которые включают хотя бы один из пяти самых дорогих товаров, доступных в нашем сервисе.
"Результат отсортируйте по возрастанию id заказа.
"Поля в результирующей таблице: order_id, product_ids"
with kak as (SELECT product_id
             FROM   (SELECT name,
                            price,
                            product_id
                     FROM   products
                     ORDER BY 2 desc limit 5) as kikik)
SELECT DISTINCT order_id,
                product_ids
FROM   (SELECT order_id,
               product_ids,
               unnest(product_ids) as productic
        FROM   orders) as kakih
WHERE  productic in (SELECT product_id
                     FROM   kak)
ORDER BY order_id asc

In [ ]:
"Используя функцию unnest, определите 10 самых популярных товаров в таблице orders.
"Самыми популярными товарами будем считать те, которые встречались в заказах чаще всего. Если товар встречается в одном заказе несколько раз (когда было куплено несколько единиц товара), это тоже учитывается при подсчёте. Учитывайте только неотменённые заказы.
"Выведите id товаров и то, сколько раз они встречались в заказах (то есть сколько раз были куплены). Новую колонку с количеством покупок товаров назовите times_purchased.
"Результат отсортируйте по возрастанию id товара.
"Поля в результирующей таблице: product_id, times_purchased"
SELECT product as product_id,
       count(product) as times_purchased
FROM   (SELECT creation_time,
               product_ids,
               unnest(product_ids) as product
        FROM   orders) as kak
GROUP BY product
ORDER BY 2 desc limit 10

In [ ]:
"Выясните, какие пары товаров покупают вместе чаще всего.
"Пары товаров сформируйте на основе таблицы с заказами. Отменённые заказы не учитывайте. В качестве результата выведите две колонки — колонку с парами наименований товаров и колонку со значениями, показывающими, сколько раз конкретная пара встретилась в заказах пользователей. Колонки назовите соответственно pair и count_pair.
"Пары товаров должны быть представлены в виде списков из двух наименований. Пары товаров внутри списков должны быть отсортированы в порядке возрастания наименования. Результат отсортируйте сначала по убыванию частоты встречаемости пары товаров в заказах, затем по колонке pair — по возрастанию.
"Поля в результирующей таблице: pair, count_pair"
with test as (SELECT spisochek.order_id,
                     product_ids,
                     name
              FROM   (SELECT order_id,
                             product_ids,
                             unnest(product_ids) as product_id
                      FROM   orders) as spisochek
                  LEFT JOIN products
                      ON products.product_id = spisochek.product_id
              WHERE  spisochek.order_id not in (SELECT order_id
                                                FROM   user_actions
                                                WHERE  action = 'cancel_order'))
SELECT pair,
       count(pair) as count_pair
FROM   (SELECT DISTINCT t.order_id,
                        array[t.name,
                        k.name] as pair
        FROM   test t
            LEFT JOIN test k
                ON t.name != k.name and
                   t.order_id = k.order_id and
                   t.name < k.name
        WHERE  k.name is not null
        GROUP BY t.order_id, t.name, k.name) as kik
GROUP BY pair
ORDER BY 2 desc, pair asc

In [ ]:
"Выясните, кто заказывал и доставлял самые большие заказы. Самыми большими считайте заказы с наибольшим числом товаров.
"Выведите id заказа, id пользователя и id курьера. Также в отдельных колонках укажите возраст пользователя и возраст курьера. Возраст измерьте числом полных лет, как мы делали в прошлых уроках. Считайте его относительно последней даты в таблице user_actions — как для пользователей, так и для курьеров. Колонки с возрастом назовите user_age и courier_age. Результат отсортируйте по возрастанию id заказа.
"Поля в результирующей таблице: order_id, user_id, user_age, courier_id, courier_age"
with orderics as (SELECT order_id,
                         courier_actions.courier_id,
                         age((SELECT max(time)
                       FROM   user_actions), birth_date) as courier_age
                  FROM   courier_actions
                      LEFT JOIN couriers
                          ON couriers.courier_id = courier_actions.courier_id
                  WHERE  action = 'deliver_order'), useruks as (SELECT order_id,
                                                     user_actions.user_id,
                                                     age((SELECT max(time)
                                                   FROM   user_actions), birth_date) as user_age
                                              FROM   user_actions
                                                  LEFT JOIN users
                                                      ON users.user_id = user_actions.user_id
                                              WHERE  action = 'create_order')
SELECT order_id,
       user_id,
       user_age::decimal,
       courier_id,
       courier_age::decimal
FROM   (SELECT dlinalosh.order_id,
               useruks.user_id,
               date_part('year', user_age)::varchar as user_age,
               orderics.courier_id,
               date_part('year', courier_age)::varchar as courier_age,
               dlina
        FROM   (SELECT order_id,
                       product_ids,
                       array_length(product_ids, 1) as dlina
                FROM   orders) as dlinalosh
            LEFT JOIN orderics
                ON dlinalosh.order_id = orderics.order_id
            LEFT JOIN useruks
                ON useruks.order_id = dlinalosh.order_id
        ORDER BY dlina desc limit 5) as orden
ORDER BY 1 asc

In [ ]:
"Произведите замену списков с id товаров из таблицы orders на списки с наименованиями товаров. Наименования возьмите из таблицы products. Колонку с новыми списками наименований назовите product_names.
"Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.
"Поля в результирующей таблице: order_id, product_names"
SELECT order_id,
       array_agg(name) as product_names
FROM   (SELECT order_id,
               product_ids,
               unnest(product_ids) as product_id
        FROM   orders) as kaka
    LEFT JOIN products
        ON products.product_id = kaka.product_id
GROUP BY order_id limit 1000

In [ ]:
"С помощью оконной функции рассчитайте медианную стоимость всех заказов из таблицы orders, оформленных в нашем сервисе. В качестве результата выведите одно число. Колонку с ним назовите median_price. Отменённые заказы не учитывайте.
"Поле в результирующей таблице: median_price"
WITH main_table AS ( SELECT order_price, ROW_NUMBER() OVER (ORDER BY order_price) AS row_number, COUNT(*) OVER() AS total_rows FROM ( SELECT SUM(price) AS order_price FROM ( SELECT order_id, product_ids, UNNEST(product_ids) AS product_id FROM orders WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action='cancel_order') ) t3 LEFT JOIN products USING(product_id) GROUP BY order_id ) t1 ) SELECT AVG(order_price) AS median_price FROM main_table WHERE row_number BETWEEN total_rows/2.0 AND total_rows/2.0 + 1

In [ ]:
"Для каждой записи в таблице user_actions с помощью оконных функций и предложения FILTER посчитайте, сколько заказов сделал и сколько отменил каждый пользователь на момент совершения нового действия.
"Иными словами, для каждого пользователя в каждый момент времени посчитайте две накопительные суммы — числа оформленных и числа отменённых заказов. Если пользователь оформляет заказ, то число оформленных им заказов увеличивайте на 1, если отменяет — увеличивайте на 1 количество отмен.
"Колонки с накопительными суммами числа оформленных и отменённых заказов назовите соответственно created_orders и canceled_orders. На основе этих двух колонок для каждой записи пользователя посчитайте показатель cancel_rate, т.е. долю отменённых заказов в общем количестве оформленных заказов. Значения показателя округлите до двух знаков после запятой. Колонку с ним назовите cancel_rate.
"В результате у вас должны получиться три новые колонки с динамическими показателями, которые изменяются во времени с каждым новым действием пользователя.
"В результирующей таблице отразите все колонки из исходной таблицы вместе с новыми колонками. Отсортируйте результат по колонкам user_id, order_id, time — по возрастанию значений в каждой.
"Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.
"Поля в результирующей таблице:
"user_id, order_id, action, time, created_orders, canceled_orders, cancel_rate"
SELECT user_id,
       order_id,
       action,
       time,
       created_orders,
       canceled_orders,
       round(canceled_orders/created_orders::decimal, 2) as cancel_rate
FROM   (SELECT user_id,
               order_id,
               action,
               time,
               count(order_id) filter(WHERE action = 'create_order') OVER(PARTITION BY user_id
                                                                          ORDER BY time asc) as created_orders,
               count(order_id) filter(WHERE action = 'cancel_order') OVER(PARTITION BY user_id
                                                                          ORDER BY time asc) as canceled_orders
        FROM   user_actions) as kak
ORDER BY user_id, order_id, action, time asc limit 1000

2 МОДУЛЬ


In [ ]:
"По таблицам orders и user_actions для каждого дня рассчитайте следующие показатели:
"Накопленную выручку на пользователя (Running ARPU).
"Накопленную выручку на платящего пользователя (Running ARPPU).
"Накопленную выручку с заказа, или средний чек (Running AOV).
"Колонки с показателями назовите соответственно running_arpu, running_arppu, running_aov. Колонку с датами назовите date.
"При расчёте всех показателей округляйте значения до двух знаков после запятой.
"Результат должен быть отсортирован по возрастанию даты.
"Поля в результирующей таблице: date, running_arpu, running_arppu, running_aov
with asa as (SELECT date,
                    order_id,
                    sum(price) as cena
             FROM   (SELECT creation_time::date as date,
                            order_id,
                            unnest(product_ids) as product_id
                     FROM   orders) as orderok
                 INNER JOIN products using(product_id)
             WHERE  order_id not in (SELECT order_id
                                     FROM   user_actions
                                     WHERE  action = 'cancel_order')
             GROUP BY date, order_id), oko as(SELECT date,
                                        sum(cena) as cena
                                 FROM   asa
                                 GROUP BY date), chislo_ordera as (SELECT date,
                                         max(orders_count) as test
                                  FROM   (SELECT date,
                                                 count(order_id) OVER (ORDER BY date) as orders_count
                                          FROM   (SELECT creation_time::date as date,
                                                         order_id,
                                                         unnest(product_ids) as product_id
                                                  FROM   orders) as orderok
                                              INNER JOIN products using(product_id)
                                          WHERE  order_id not in (SELECT order_id
                                                                  FROM   user_actions
                                                                  WHERE  action = 'cancel_order')
                                          GROUP BY date, order_id) as koko
                                  GROUP BY date), total as (SELECT DISTINCT date,
                                          count(user_id) OVER (ORDER BY date) as total_users
                          FROM   (SELECT user_id,
                                         min(time)::date as date
                                  FROM   user_actions
                                  GROUP BY user_id) as koko
                          ORDER BY 1 asc), tolko_plati as ((SELECT DISTINCT date,
                                                  count(user_id) OVER (ORDER BY date) as plati
                                  FROM   (SELECT user_id,
                                                 min(time)::date as date
                                          FROM   user_actions
                                          WHERE  order_id not in (SELECT order_id
                                                                  FROM   user_actions
                                                                  WHERE  action = 'cancel_order')
                                          GROUP BY user_id) as koko
                                  ORDER BY 1 asc))
SELECT date,
       round(sum(cena) OVER(ORDER BY date asc)/total_users, 2) as running_arpu,
       round(sum(cena) OVER(ORDER BY date asc)/plati, 2) as running_arppu,
       round(sum(cena) OVER(ORDER BY date asc)/test, 2) as running_aov
FROM   oko
    LEFT JOIN total using(date)
    LEFT JOIN tolko_plati using(date)
    LEFT JOIN chislo_ordera using(date)

In [ ]:
"Для каждого дня в таблицах orders и user_actions рассчитайте следующие показатели:
"Выручку, полученную в этот день.
"Выручку с заказов новых пользователей, полученную в этот день.
"Долю выручки с заказов новых пользователей в общей выручке, полученной за этот день.
"Долю выручки с заказов остальных пользователей в общей выручке, полученной за этот день.
"Колонки с показателями назовите соответственно revenue, new_users_revenue, new_users_revenue_share, old_users_revenue_share. Колонку с датами назовите date.
"Все показатели долей необходимо выразить в процентах. При их расчёте округляйте значения до двух знаков после запятой.
"Результат должен быть отсортирован по возрастанию даты.
"Поля в результирующей таблице:
"date, revenue, new_users_revenue, new_users_revenue_share, old_users_revenue_share
with chech as (SELECT order_id,
                      sum(price) as order_price
               FROM   (SELECT creation_time,
                              order_id,
                              unnest(product_ids) as product_id
                       FROM   orders
                       WHERE  order_id not in (SELECT order_id
                                               FROM   user_actions
                                               WHERE  action = 'cancel_order')) as t
                   LEFT JOIN products using (product_id)
               GROUP BY order_id), aka as (SELECT date,
                                   revenue
                            FROM   (SELECT creation_time::date as date,
                                           sum(price) as revenue
                                    FROM   (SELECT creation_time,
                                                   unnest(product_ids) as product_id
                                            FROM   orders
                                            WHERE  order_id not in (SELECT order_id
                                                                    FROM   user_actions
                                                                    WHERE  action = 'cancel_order')) t1
                                        LEFT JOIN products using (product_id)
                                    GROUP BY date) t2), iko as (SELECT test.date,
                                   revenue,
                                   sum(order_price) as new_revenue
                            FROM   (SELECT time::date as date,
                                           user_id,
                                           order_id,
                                           order_price
                                    FROM   user_actions
                                        INNER JOIN chech using(order_id))as test
                                INNER JOIN (SELECT user_id,
                                                   min(time::date) as date
                                            FROM   user_actions
                                            GROUP BY user_id
                                            ORDER BY 1 asc) as oko
                                    ON oko.user_id = test.user_id and
                                       oko.date = test.date
                                INNER JOIN aka
                                    ON test.date = aka.date
                            GROUP BY test.date, revenue)
SELECT date,
       revenue,
       new_revenue as new_users_revenue,
       round(new_revenue*100/revenue::decimal, 2) as new_users_revenue_share,
       round(100-(new_revenue*100/revenue::decimal), 2) as old_users_revenue_share
FROM   iko

In [ ]:
"Для каждой рекламной кампании для каждого дня посчитайте две метрики:
"1. Накопительный ARPPU.
"2. Затраты на привлечение одного покупателя (CAC).
"Колонку с наименованиями кампаний назовите ads_campaign, колонку с днями — day, а колонки со значениями метрик — cumulative_arppu и cac.
"Значения метрики CAC укажите одинаковым для всех дней (это необходимо для наглядной визуализации).
"Наименования кампаний выведите в следующем виде:
"Кампания № 1
"Кампания № 2
"Дни пронумеруйте начиная с 0 и отобразите в следующем формате:
"Day 0
"Day 1
"Day 2
"и т.д.
"Полученные значения метрик необходимо округлить до двух знаков после запятой.
"Результат должен быть отсортирован сначала по наименованию кампании (по возрастанию), затем по наименованию дня (также по возрастанию).
"Поля в результирующей таблице: ads_campaign, day, cumulative_arppu, cac
with oko as (SELECT date,
                    order_id,
                    user_id
             FROM   (SELECT DISTINCT time::date as date,
                                     user_id,
                                     order_id
                     FROM   user_actions
                     WHERE  order_id not in (SELECT order_id
                                             FROM   user_actions
                                             WHERE  action = 'cancel_order')) as kok), lol as (SELECT date,
                                                         sum(order_price) as revenue,
                                                         (SELECT count(distinct user_id)
                                                   FROM   oko
                                                   WHERE  user_id in (8804, 9828, 9524, 9667, 9165, 10013, 9625, 8879, 9145, 8657, 8706, 9476, 9813, 8940, 9971, 10122, 8732, 9689, 9198, 8896, 8815, 9689, 9075, 9071, 9528, 9896, 10135, 9478, 9612, 8638, 10057, 9167, 9450, 9999, 9313, 9720, 9599, 9351, 8638, 8752, 9998, 9431, 9660, 9004, 8632, 8896, 8750, 9605, 8867, 9535, 9494, 9762, 8990, 9526, 9786, 9654, 9144, 9391, 10016, 8988, 9009, 9061, 9004, 9550, 8707, 8788, 8988, 8853, 9836, 8810, 9916, 9660, 9677, 9896, 8933, 8828, 9108, 9180, 9897, 9960, 9472, 9818, 9695, 9965, 10023, 8972, 9035, 8869, 9662, 9561, 9740, 8723, 9146, 10103, 9963, 10103, 8715, 9167, 9313, 9679, 9251, 10001, 8867, 8707, 9945, 9562, 10013, 9020, 9317, 9002, 9838, 9144, 8911, 9505, 9313, 10134, 9197, 9398, 9652, 9999, 9210, 8741, 9963, 9422, 9778, 8815, 9512, 9794, 9019, 9287, 9561, 9321, 9677, 10122, 8752, 9810, 9871, 9162, 8876, 9414, 10030, 9334, 9175, 9182, 9451, 9257, 9321, 9531, 9655, 9845, 8883, 9993, 9804, 10105, 8774, 8631, 9081, 8845, 9451, 9019, 8750, 8788, 9625, 9414, 10064, 9633, 9891, 8751, 8643, 9559, 8791, 9518, 9968, 9726, 9036, 9085, 9603, 8909, 9454, 9739, 9223, 9420, 8830, 9615, 8859, 9887, 9491, 8739, 8770, 9069, 9278, 9831, 9291, 9089, 8976, 9611, 10082, 8673, 9113, 10051)) as tost
                                                  FROM   (SELECT order_id,
                                                                 sum(price) as order_price
                                                          FROM   (SELECT order_id,
                                                                         creation_time,
                                                                         unnest(product_ids) as product_id
                                                                  FROM   orders
                                                                  WHERE  order_id not in (SELECT order_id
                                                                                          FROM   user_actions
                                                                                          WHERE  action = 'cancel_order')) as ll
                                                              LEFT JOIN products
                                                                  ON ll.product_id = products.product_id
                                                          GROUP BY order_id) as koko
                                                      LEFT JOIN oko
                                                          ON koko.order_id = oko.order_id
                                                  WHERE  user_id in (8804, 9828, 9524, 9667, 9165, 10013, 9625, 8879, 9145, 8657, 8706, 9476, 9813, 8940, 9971, 10122, 8732, 9689, 9198, 8896, 8815, 9689, 9075, 9071, 9528, 9896, 10135, 9478, 9612, 8638, 10057, 9167, 9450, 9999, 9313, 9720, 9599, 9351, 8638, 8752, 9998, 9431, 9660, 9004, 8632, 8896, 8750, 9605, 8867, 9535, 9494, 9762, 8990, 9526, 9786, 9654, 9144, 9391, 10016, 8988, 9009, 9061, 9004, 9550, 8707, 8788, 8988, 8853, 9836, 8810, 9916, 9660, 9677, 9896, 8933, 8828, 9108, 9180, 9897, 9960, 9472, 9818, 9695, 9965, 10023, 8972, 9035, 8869, 9662, 9561, 9740, 8723, 9146, 10103, 9963, 10103, 8715, 91